### Embed data from eventalign for plotting
Check to see if alignments from eventalign match signalalign

In [33]:
import os
import pandas as pd
import numpy as np
from signalalign.fast5 import Fast5


In [56]:
nanopolish_data_dir = "/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/nanopolish"
nanopolish_readdb_dir = "/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/fastq_nanopolish"

key_names = ["cbfgal","cbfglu","ivt","nopgal","nopglu"]
nanopolish_data_paths = [os.path.join(nanopolish_data_dir, name)+"_eventalign.tsv" for name in key_names]
readdb_paths = [os.path.join(nanopolish_readdb_dir, name)+".fastq.index.readdb" for name in key_names]

print(nanopolish_data_paths)
print(readdb_paths)

['/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/nanopolish/cbfgal_eventalign.tsv', '/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/nanopolish/cbfglu_eventalign.tsv', '/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/nanopolish/ivt_eventalign.tsv', '/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/nanopolish/nopgal_eventalign.tsv', '/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/nanopolish/nopglu_eventalign.tsv']
['/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/fastq_nanopolish/cbfgal.fastq.index.readdb', '/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/fastq_nanopolish/cbfglu.fastq.index.readdb', '/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/fastq_nanopolish/ivt.fastq.index.readdb', '/Users/andrewbailey/CLionProjects/per

In [57]:
def get_fast5_paths(read_db_path):
    fast5_files = []
    with open(read_db_path, 'r') as fofn:
        for line in fofn:
            fast5_files.append(line.split('\t')[1][:-1])
    return fast5_files


In [58]:
nanopolish_data = [pd.read_csv(x, sep="\t") for x in nanopolish_data_paths]
nanopolish_f5_paths = [get_fast5_paths(x) for x in readdb_paths]


In [70]:
dtype = [('contig', 'S10'), 
        ('position', int),
        ('reference_kmer', 'S6'), 
        ('read_name', "S50"),
        ('strand', 'S1'), 
        ('event_index', int),
        ('event_level_mean', float), 
        ('event_stdv', float),
        ('event_length', float), 
        ('model_kmer', 'S6'),
        ('model_mean', float), 
        ('model_stdv', float),
        ('standardized_level', float),
        ('start_idx', int),
        ('end_idx', int)]
dtype_dict = {'contig': 'S10', 
                'position': int,
                'reference_kmer': 'S6',
                'read_name': 'S50',
                'strand': 'S1', 
                'event_index': int,
                'event_level_mean': float,
                'event_stdv': float,
                'event_length': float,
                'model_kmer': 'S6',
                'model_mean': float,
                'model_stdv': float,
                'standardized_level': float,
                'start_idx': int,
                'end_idx': int}

for data, f5_paths in zip(nanopolish_data, nanopolish_f5_paths):
    for index, y in data.groupby(["read_name"]):
        read_id = index.split(":")[0]
        template = None
        complement = None
        for strand, data  in y.groupby("strand"):
            if strand == "t":
                template = data.to_records(index=False, column_dtypes=dtype_dict)
            else:
                complement = data.to_records(index=False, column_dtypes=dtype_dict)
        fast5_file = [x for x in f5_paths if read_id in x][0]
        handle = Fast5(fast5_file, read='r+')
        handle.set_eventalign_table(template=template, complement=complement, meta=None, overwrite=True)
        handle.close()
        

['/Users/andrewbailey/CLionProjects/personal/projects/ares_rRNA/small_local_sample/fast5/cbfgal/read_4eea606b-07bf-4ef5-858e-ef932ff03e56.fast5']